In [1]:
from evaluations import *
acc = 1e-2
boundary = 'robin'
krang  = np.arange(0.1,10.1,0.1)
cloc0 = 0
cloc1 = 1
cloc2 = 1
coarse_grid_resolution = 10
p = helmholtz(boundary = boundary)
k = 6.
cglob = -1j*k
cloc = cloc0+ cloc1*k+cloc2*k**2
mus = {'k': k, 'c_glob': cglob, 'c_loc': cloc}
resolution  = int(np.ceil(float(k*1.5+50)/coarse_grid_resolution)*coarse_grid_resolution)
gq, lq = localize_problem(p, coarse_grid_resolution, resolution, mus = mus, calT = True, calQ = True)
calculate_continuity_constant(gq, lq)
calculate_inf_sup_constant2(gq, lq)	
calculate_lambda_min(gq, lq)
calculate_csis(gq,lq)	
calculate_Psi_norm(gq,lq)
d = gq["d"]
u = d.solve(mus)
norm = gq["full_norm"]
localizer = gq["localizer"]
pou = gq["pou"]
bases = create_bases(gq, lq, num_testvecs=20, transfer = 'robin', target_accuracy = acc, calC = False)
ru = reconstruct_solution(gq,lq,bases)
ls = norm(u-ru)[0]/norm(u)[0]
sum1 = NumpyVectorArray(u.data*0)
sum2 = 0
sum3 = 0
sum4 = 0
sum51 = 0
sum52 = 0
sum6 = 0
for space in gq["spaces"]:
    ldict = lq[space]
    basis = bases[space]
    M = ldict["range_product"]._matrix
    S = ldict["source_product"]._matrix
    M_sparse = scipy.sparse.csr_matrix(M)
    T = ldict["transfer_matrix_robin"]
    B = basis._array.T
    range_space = ldict["range_space"]
    omega_star_space = ldict["omega_star_space"]
    u_s = ldict["local_solution_robin"]
    u_s2 = ldict["local_sol2"]
    local_norm1 = induced_norm(ldict["range_product"])
    local_norm2 = induced_norm(ldict["omega_star_product"])

    u_loc = pou[range_space](localizer.localize_vector_array(u, range_space))
    u_loc2 = localizer.localize_vector_array(u, omega_star_space)
    u_dif = u_loc-u_s
    u_dif2 = u_loc2-u_s2
    term1 = u_dif.data.T - B.dot(B.conj().T).dot(M_sparse.dot(u_dif.data.T))
    term2 = NumpyVectorArray(term1.T)

    T1 = T - B.dot(B.conj().T).dot(M_sparse.dot(T))
    maxval = operator_svd2(T1, S, M_sparse)[0][0]

    sum1 += localizer.globalize_vector_array(term2, range_space)
    sum2 += local_norm1(term2)[0]
    sum3 += maxval*ldict["Psi_norm"]*local_norm2(u_dif2)[0]
    sum4 += maxval*ldict["Psi_norm"]*local_norm2(u_loc2)[0]**(1./2.)*ldict["csi"]
    sum51 += maxval**2*ldict["Psi_norm"]**2*ldict["csi"]**2
    sum52 += local_norm2(u_loc2)[0]**2
rs1 = gq["continuity_constant"]/gq["inf_sup_constant"]* norm(sum1)[0]
rs2 = gq["continuity_constant"]/gq["inf_sup_constant"]* sum2
rs3 = gq["continuity_constant"]/gq["inf_sup_constant"]* sum3
rs4 = gq["continuity_constant"]/gq["inf_sup_constant"]* sum4
rs5 = gq["continuity_constant"]/gq["inf_sup_constant"]* np.sqrt(sum51) * np.sqrt(sum52)
rs6 = gq["continuity_constant"]/gq["inf_sup_constant"]* np.sqrt(sum51) *4 * norm(u)[0]
ccs = gq["continuity_constant"]/gq["inf_sup_constant"]

Loading pyMOR version 0.4.2-116-gadb475af
localizing problem
calculated_continuity_constant:  1.791868186723067
calculated_inf_sup_constant:  0.22732564927664775
calculated all lambdas


constants.py:137: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  x = np.linalg.lstsq(T, u_s.data.T)[0]
/home/root_dennis/pymor_MA/src/pymor/operators/constructions.py:1002: RuntimeWarning: invalid value encountered in double_scalars
  if (np.linalg.norm(norm_squared.imag)/np.linalg.norm(norm_squared) > 1e-11):
/usr/lib/python2.7/dist-packages/scipy/sparse/linalg/eigen/arpack/arpack.py:737: ComplexWarning: Casting complex values to real discards the imaginary part
  self.workd[yslice] = self.OP(self.workd[xslice])
/usr/lib/python2.7/dist-packages/scipy/sparse/linalg/eigen/arpack/arpack.py:746: ComplexWarning: Casting complex values to real discards the imaginary part
  self.workd[yslice] = self.B(self.workd[xslice])
/usr/lib/python2.7/dist-packages/scipy

calculated csis
calculated all Psi_norms


mybmat.py:20: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  if np.rank(blocks) != 2:


In [2]:
print rs1, rs2, rs3, rs4, rs5, rs6, ccs

1.4671897591300591e-05 0.00012055407239994782 0.00023786668359948403 0.0002703765909701086 0.00030225159299289005 0.0003639399538854921 7.882384554601769


In [2]:
print rs1, rs2, rs3, rs4, rs5, rs6, ccs

0.01805190019609898 0.14621843608764062 0.19549607767945512 0.22115034330135522 0.2520350164243034 0.3034743716210882 7.882384554601769


In [1]:
1

1

In [2]:
d

NameError: name 'd' is not defined

In [3]:
diameter = 1./resolution
d, data = discretize_elliptic_cg(p, diameter=diameter)

In [4]:
u = d.solve(mus)
rhs = NumpyVectorArray(d.rhs.assemble(mus)._matrix)

In [11]:
for k in np.arange(0.1,10.1,0.1):
    cglob = -1j*k
    mus = {'k': k, 'c_glob': cglob, 'c_loc': cloc}

    u = d.solve(mus)
    rhs = NumpyVectorArray(d.rhs.assemble(mus)._matrix)

    a = d.h1_norm(u)


    op = d.operator.assemble(mus)
    A = op._matrix
    H1 = d.h1_product.assemble(mus)._matrix
    Y = H1
    X = H1

    Yinv = sp.factorized(Y.astype(complex))
    def mv(v):
        return A.H.dot(Yinv(A.dot(v)))
    M1 = LinearOperator(A.shape, matvec = mv)

    #t = time.time()
    eigvals = sp.eigs(M1, M=X, which = 'SM', tol = 1e-1, k=100)[0]
    #print time.time()-t
    eigvals = np.sqrt(np.abs(eigvals))
    eigvals.sort()
    result = eigvals[0]

    b = d.l2_norm(rhs)/result
    print a, b, b/a

[0.08044871] [6.60281245e-05] [0.00082075]
[0.04515947] [3.49481909e-05] [0.00077388]
[0.03484629] [2.93181743e-05] [0.00084136]
[0.03034278] [2.86124215e-05] [0.00094297]
[0.02792647] [2.85102886e-05] [0.00102091]
[0.02643141] [2.85715423e-05] [0.00108097]
[0.025399] [2.87121274e-05] [0.00113044]
[0.02462194] [2.89062417e-05] [0.001174]
[0.02399721] [2.91436996e-05] [0.00121446]
[0.02347007] [2.94198394e-05] [0.0012535]
[0.02300987] [2.97323309e-05] [0.00129216]
[0.02259882] [3.00799562e-05] [0.00133104]
[0.02222618] [3.0462076e-05] [0.00137055]
[0.02188529] [3.08783684e-05] [0.00141092]
[0.02157184] [3.13286883e-05] [0.0014523]
[0.02128294] [3.18129834e-05] [0.00149476]
[0.02101656] [3.23312387e-05] [0.00153837]
[0.02077115] [3.28834346e-05] [0.00158313]
[0.02054551] [3.34695129e-05] [0.00162904]
[0.02033862] [3.40893452e-05] [0.00167609]
[0.02014961] [3.47427019e-05] [0.00172424]
[0.01997767] [3.54292211e-05] [0.00177344]
[0.01982208] [3.6148376e-05] [0.00182364]
[0.01968215] [3.689

KeyboardInterrupt: 

In [12]:
d.h1_norm(rhs)

array([0.00199468])

In [13]:
d.l2_norm(rhs)

array([2.618914e-05])

In [8]:
b

array([6.32484518e-05])

In [9]:
result

0.4140676854476648

In [14]:
d.visualize(rhs)